In [1]:
from github import Github
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd 
import numpy as np 

In [2]:
# Set up github access and get members from an organization
pd.set_option('display.max_rows', None)
g = Github("6dec4a753453cdbf66d526d8a646004deea5d11c")
df = pd.DataFrame(columns=['Member'])
memberList = []
for member in g.get_organization("PyGithub").get_members():
    memberList.append(member.login)

In [3]:
# Get all of the member's repositories
df = pd.DataFrame(columns=['Username','Repository','DateCreated','Language', 'Size'])
for member in memberList:
    currentUser = g.get_user(member)
    for repo in currentUser.get_repos():
        new_row = {'Username': member.strip(), 'Repository': repo.name.strip(), 'DateCreated': repo.created_at.year, 'Language':repo.language, 'Size':repo.size, 'Contributors':repo.get_contributors().totalCount}
        df = df.append(new_row, ignore_index=True)

In [4]:
df = df.sort_values(by=['Username', 'DateCreated'])
df = df[(df[['Size','Contributors']] != 0).all(axis=1)]
df = df[df.Language != 'None']
for member in memberList:
    mask = (df['Username'] == member)
    df['DateCreated'][mask] = df['DateCreated'] - (df.loc[df['Username'] == member])['DateCreated'].min()

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
tempArray = df.loc[df['DateCreated'] == 3]
tempArray = tempArray.drop(['Username', 'Repository',"DateCreated"], axis=1)
yearArray = pd.DataFrame(columns=['Language','Repositories','Average Size','Contributors'])
yearArray['Language'] = tempArray.Language.unique()
uniqueValues = tempArray.Language.unique()
for language in uniqueValues:
    mask = (yearArray['Language'] == language)
    yearArray['Repositories'][mask] = tempArray.loc[tempArray['Language'] == language].Language.count()
    yearArray['Average Size'][mask] = tempArray.loc[tempArray['Language'] == language].Size.mean()
    yearArray['Contributors'][mask] = tempArray.loc[tempArray['Language'] == language].Contributors.mean()
yearArray = yearArray.replace(to_replace='None', value=np.nan).dropna()
yearArray

,Language,Repositories,Average Size,Contributors
0,Python,18.0,4117.277778,30.222222
2,Shell,4.0,46.500000,1.750000
3,JavaScript,5.0,1474.600000,9.400000
4,Java,1.0,1984.000000,4.000000


In [10]:
fig = px.scatter(x=yearArray['Repositories'], y=yearArray['Contributors'], size = yearArray['Average Size'], color=yearArray['Average Size'])
fig.show()